# ToDo
* p=1~20までの全動画を作成する
* s=1000までデータを取って作る

# 動かし方
* 「必要な変数の定義」と「C++による計算」のパラメータを調整する
* 「画像を作る」まで全部実行
* ランタイムを再起動
* 「ライブラリのインポート」「必要な変数の定義」→「フォルダ内のpngをソート」→「前半のGIFを作成」を実行
* ランタイムを再起動
* 「ライブラリのインポート」「必要な変数の定義」→「フォルダ内のpngをソート」→「後半のGIFを作成」以降を実行




### ライブラリのインポート

In [1]:
# インポート系
!pip install pillow
from PIL import Image, ImageDraw, ImageFont
from collections import defaultdict
from sys import setrecursionlimit
from copy import deepcopy
import time
from functools import lru_cache
import sys
from IPython.utils.py3compat import u_format
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap
import imageio
import os
import csv
import subprocess

# ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

# 準備
setrecursionlimit(10**8)  # 再帰計算の上限回数を上げる
def LL(h,w,e=0): return [[e for i in range(w)] for j in range(h)]
def LLL(a,b,c,e=0): return [LL(b,c,e) for _ in range(a)]
def LLLL(a,b,c,d,e=0): return [LLL(b,c,d,e) for _ in range(a)]

Mounted at /content/drive


### 必要な変数の定義

In [3]:
# 変数名定義など

p = 5  # 禁止手数
s = 300  # 探索範囲
ver = 10

csv_path = "/content/drive/MyDrive/result_sbnim.csv"  # csvファイル
png_dir = '/content/drive/MyDrive/研究 組合せ/３次元スライスp={}, s={} ({})/'.format(p,s,ver)  # pngファイルの置き場所
gif_filename_1 = 'output_part1.gif'
gif_filename_2 = 'output_part2.gif'
gif_filename_final = 'p={}.gif'.format(p)

colormap = plt.get_cmap('hsv')  # カラーマップ
width, height = 3200, 2400
margin = 50  # 余白のサイズ

# 画像にステータスを書き込む
font_path = '/usr/share/fonts/truetype/liberation/LiberationSerif-BoldItalic.ttf'
font_size = 60
status_position = (2600,100)
font = ImageFont.truetype(font_path, font_size)

# 該当フォルダがない場合、新しくフォルダを作成する
if not os.path.exists(png_dir):
    os.makedirs(png_dir)


### C++による計算
pとsの値を変更する

In [ ]:
%%writefile /content/drive/MyDrive/calc_sbnim.cpp
#include <bits/stdc++.h>
using namespace std;

template<class T> inline bool chmax(T& a, T b) { if (a < b) { a = b; return 1; } return 0; }
template<class T> inline bool chmin(T& a, T b) { if (a > b) { a = b; return 1; } return 0; }
typedef long long ll;
#define rep(i,n) for(int i=0;i<(int)(n);i++)
#define rep1(i,n) for(int i=1;i<(int)(n);i++)
#define replr(i,l,r) for(int i=l;i<(int)(r);i++)
#define All(c) (c).begin(), (c).end()
#define Print(x) cout<<(x)<<endl
#define Prints(x) cout<<setprecision(10)<<(x)<<endl
#define Printl(A,n) rep(i,n) {cout<<A[i]<<' ';} cout<<endl;


int p;  // 禁止手数
int s;  // 見る範囲
int f[1010][1010][1010];  // dp配列
int d[100];  // p<=80ぐらいでやるならこれでよい

int g(int x, int y, int z) {
    if (f[x][y][z] <= p) return 1;
    else return 0;
}

int main() {

    ofstream outfile("/content/drive/MyDrive/result_sbnim.csv");

    p = 5;  // 禁止手数
    s = 300;  // サイズ

    // 初期値1
    rep(i, s) {
        f[i][0][0] = min(i, p+1);
        f[0][i][0] = min(i, p+1);
        f[0][0][i] = min(i, p+1);
    }

    // 初期値2
    rep1(i, s) {
        rep1(j, s) {
            int v = f[i-1][j][0] + f[i][j-1][0] - f[i-1][j-1][0] - 2*g(i-1,j-1,0) + g(i-1,j,0) + g(i,j-1,0);
            f[i][j][0] = v;
            f[i][0][j] = v;
            f[0][i][j] = v;
        }
    }

    // 計算
    rep1(x,s) {
        rep1(y,s) {
            rep1(z,s) {
                f[x][y][z] = f[x-1][y][z] + f[x][y-1][z] + f[x][y][z-1]
                        + f[x-1][y-1][z-1]
                        - (f[x][y-1][z-1] + f[x-1][y][z-1] + f[x-1][y-1][z])
                        + 3 * g(x-1,y-1,z-1)
                        - 2 * (g(x-1,y-1,z) + g(x-1,y,z-1) + g(x,y-1,z-1))
                        + g(x,y,z-1) + g(x,y-1,z) + g(x-1,y,z);
            }
        }
    }

    // レンジと個数の確認
    int xl = 5000;
    int xr = 0;

    rep(x,s) {
        replr(y,x,s) {
            replr(z,y,s) {
                if (x+y+z>=p) {
                    chmax(xr, f[x][y][z]);
                    chmin(xl, f[x][y][z]);
                    d[f[x][y][z]] += 1;
                }
            }
        }
    }

    Print(xl);
    Print(xr);
    Printl(d, 100);


    // データを書き出す
    rep(x,s) {
        rep(y,s) {
            rep(z,s) {
                outfile << f[x][y][z] << " ";
            }
            outfile << endl;
        }
    }

    outfile.close();


}




Overwriting /content/drive/MyDrive/calc_sbnim.cpp


In [ ]:
!g++ -mcmodel=large -o /content/drive/MyDrive/calc_sbnim /content/drive/MyDrive/calc_sbnim.cpp

In [ ]:
!/content/drive/MyDrive/calc_sbnim

2
6
0 0 397 39185 2996491 1211335 297688 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


### Pythonにデータを渡す

In [ ]:
# csvの内容を配列に格納する

f1 = []
cnt = 0
f = []

with open(csv_path, "r", newline="", encoding="utf-8") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=" ")
    for row in csvreader:
        a = []
        for i in filter(None, row):
            a.append(int(i))
        f1.append(a)

for i in range(s):
    ff = []
    for j in range(s):
        ff.append(f1[i*s + j])
    f.append(ff)

In [ ]:
# 配列を扱いやすい形にする。ここで座標の移動も済ませる。

points = [[] for i in range(3*s+1)]
xl,xr = 10**10, 0
Xm = [10000] * (3*s+1)
XM = [0] * (3*s+1)
Ym = [10000] * (3*s+1)
YM = [0] * (3*s+1)
Scale = [0]*(3*s)

for x in range(s):
    for y in range(s):
        for z in range(s):

            if x+y+z>=p:
                xl = min(xl, f[x][y][z])
                xr = max(xr, f[x][y][z])
                k = f[x][y][z]
                t = x+y+z

                X = x * (3**.5/6 + 1/2) + y * (3**.5/6 - 1/2) - 3**.5/3 * z
                Y = x * (3**.5/6 - 1/2) + y * (3**.5/6 + 1/2) - 3**.5/3 * z

                points[t].append((X,Y,k))

                Xm[t] = min(Xm[t],X)
                XM[t] = max(XM[t],X)
                Ym[t] = min(Ym[t],Y)
                YM[t] = max(YM[t],Y)

### 画像を作る

In [ ]:
print(XM[:20])
print(Xm[:20])

[0, 0.7886751345948129, 1.5773502691896257, 2.3660254037844384, 3.1547005383792515, 3.9433756729740645, 4.732050807568877, 5.52072594216369, 6.309401076758503, 7.098076211353316, 7.886751345948129, 8.675426480542942, 9.464101615137753, 10.252776749732567, 11.04145188432738, 11.830127018922193, 12.618802153517006, 13.407477288111819, 14.196152422706632, 14.984827557301445]
[0.0, -0.5773502691896257, -1.1547005383792515, -1.7320508075688772, -2.309401076758503, -2.8867513459481287, -3.4641016151377544, -4.04145188432738, -4.618802153517006, -5.196152422706632, -5.773502691896257, -6.350852961085883, -6.928203230275509, -7.505553499465135, -8.08290376865476, -8.660254037844386, -9.237604307034012, -9.814954576223638, -10.392304845413264, -10.969655114602888]


In [ ]:
# 画像作成に必要なパラメータをそろえる

for i in range(p,3*s//2+1):  # 本来はpから
    #Scale[i] = min((width - 2 * margin) / (XM[i] - Xm[i]), (height - 2 * margin) / (YM[i] - Ym[i]))
    # ここ変更
    u = 300
    Scale[i] = min((width - 2 * margin) / (XM[u] - Xm[u]), (height - 2 * margin) / (YM[u] - Ym[u]))

print(xl,xr)

L = points[xl:xr+1]
#M = len(L)
M=9  # ここ変更

print(M)

Color = []
for i in range(M):
    Color.append(tuple(int(255 * c) for c in colormap(i/M)[:3]))

2 6
9


In [ ]:
# プロットサイズを段階的に設定する
"""
PS = [0] * 1000
for i in range(1000):
    if i<50:
        PS[i] = 10
    elif i<100:
        PS[i] = 7
    elif i<200:
        PS[i] = 5
    else:
        PS[i] = 2
"""

PS = [2]*1000


In [ ]:
# 画像を生成する

for t in range(p, s*3//2+1): # p~s*3//2+1  # 本来はpから  # 普通のNimなら1~s*3//2+1に
    print(t, len(points[t]))

    image = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(image)

    # ステータス情報を追記
    status = "p={}, k={}, s={}".format(p,t,s)
    draw.text(status_position, status, fill=(0,0,0), font=font)

    for x,y,i in points[t]:
        # ここ変更
        u = 300
        plot_x = Scale[t] * (x - Xm[u]) + margin
        plot_y = Scale[t] * (YM[u] - y) + margin

        #plot_x = Scale[t] * (x - Xm[t]) + margin
        #plot_y = Scale[t] * (YM[t] - y) + margin
        rgb_color = Color[i-xl]

        ps = PS[t]

        draw.ellipse((plot_x - ps, plot_y - ps, plot_x + ps, plot_y + ps), fill=rgb_color)

    image.save(png_dir + "p={}k={}s={}.png".format(p,"{:03}".format(t),s))

3 10
4 15
5 21
6 28
7 36
8 45
9 55
10 66
11 78
12 91
13 105
14 120
15 136
16 153
17 171
18 190
19 210
20 231
21 253
22 276
23 300
24 325
25 351
26 378
27 406
28 435
29 465
30 496
31 528
32 561
33 595
34 630
35 666
36 703
37 741
38 780
39 820
40 861
41 903
42 946
43 990
44 1035
45 1081
46 1128
47 1176
48 1225
49 1275
50 1326
51 1378
52 1431
53 1485
54 1540
55 1596
56 1653
57 1711
58 1770
59 1830
60 1891
61 1953
62 2016
63 2080
64 2145
65 2211
66 2278
67 2346
68 2415
69 2485
70 2556
71 2628
72 2701
73 2775
74 2850
75 2926
76 3003
77 3081
78 3160
79 3240
80 3321
81 3403
82 3486
83 3570
84 3655
85 3741
86 3828
87 3916
88 4005
89 4095
90 4186
91 4278
92 4371
93 4465
94 4560
95 4656
96 4753
97 4851
98 4950
99 5050
100 5151
101 5253
102 5356
103 5460
104 5565
105 5671
106 5778
107 5886
108 5995
109 6105
110 6216
111 6328
112 6441
113 6555
114 6670
115 6786
116 6903
117 7021
118 7140
119 7260
120 7381
121 7503
122 7626
123 7750
124 7875
125 8001
126 8128
127 8256
128 8385
129 8515
130 8646
131

### gifを作る

In [ ]:
# フォルダ内のpngファイルのリストを取得し、名前順にソート
png_files = sorted([f for f in os.listdir(png_dir) if f.endswith('.png')])

In [ ]:
# 前半のGIFを作成
with imageio.get_writer(os.path.join(png_dir, gif_filename_1), mode='I', duration=0.3) as writer:
    for i, png_file in enumerate(png_files):  # 前半の画像を選択
        if i>200:
            break
        if i%5==0:
            print(i)
        image_path = os.path.join(png_dir, png_file)
        image = imageio.imread(image_path)
        writer.append_data(image)

0


<ipython-input-4-3584d7f55955>:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200


In [ ]:
# 後半のGIFを作成
with imageio.get_writer(os.path.join(png_dir, gif_filename_2), mode='I', duration=0.3) as writer:
    for i, png_file in enumerate(png_files):  # 後半の画像を選択
        if i<=200:
            continue
        if i%5==0:
            print(i)
        image_path = os.path.join(png_dir, png_file)
        image = imageio.imread(image_path)
        writer.append_data(image)

<ipython-input-5-07447cdcebe4>:9: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(image_path)


205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445


In [ ]:
!apt-get update
!apt-get install gifsicle -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [645 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,595 kB]
Get:13 http://archive.ubuntu.com/

In [ ]:
# あらかじめ定義されたファイルパス
gif_path_1 = os.path.join(png_dir, gif_filename_1)
gif_path_2 = os.path.join(png_dir, gif_filename_2)
final_gif_path = os.path.join(png_dir, gif_filename_final)

# コマンドを構築
gifsicle_command = f'gifsicle --loopcount=0 "{gif_path_1}" "{gif_path_2}" > "{final_gif_path}"'

# エラーメッセージをキャプチャするためにsubprocess.runを実行
proc = subprocess.run(gifsicle_command, shell=True, text=True, capture_output=True)

# 成功したかどうかをチェック
if proc.returncode != 0:
    print("gifsicle command failed.")
    print("Error message:", proc.stderr)
else:
    print("GIF created successfully.")


GIF created successfully.


# 縦スライス画像を作る

In [ ]:
# csvの内容を配列に格納する

f1 = []
cnt = 0
f = []

with open(csv_path, "r", newline="", encoding="utf-8") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=" ")
    for row in csvreader:
        a = []
        for i in filter(None, row):
            a.append(int(i))
        f1.append(a)

for i in range(s):
    ff = []
    for j in range(s):
        ff.append(f1[i*s + j])
    f.append(ff)

In [ ]:
# 配列を扱いやすい形にする。ここで座標の移動も済ませる。

points = [[] for i in range(3*s+1)]
xl,xr = 10**10, 0
Xm = [10000] * (3*s+1)
XM = [0] * (3*s+1)
Ym = [10000] * (3*s+1)
YM = [0] * (3*s+1)

for x in range(s):
    for y in range(s):
        for z in range(s):

            if x+y+z>=p:
                xl = min(xl, f[x][y][z])
                xr = max(xr, f[x][y][z])
                k = f[x][y][z]
                t = x+y+z

                points[z].append((x,y,k))

画像

In [ ]:
L = points[xl:xr+1]
M = len(L)
Scale = 7
ps= 3

print(M)

Color = []
for i in range(M):
    Color.append(tuple(int(255 * c) for c in colormap(i/M)[:3]))

# 画像を生成する

for z in range(40,s):

    image = Image.new("RGB", (width, height), "white")
    draw = ImageDraw.Draw(image)

    # ステータス情報を追記
    status = "p={}, z={}".format(p,z)
    draw.text(status_position, status, fill=(0,0,0), font=font)

    for x,y,i in points[z]:
        # plot_x = Scale * (x - 100) + margin
        # plot_y = Scale * (100 - y) + margin
        rgb_color = Color[i-xl]
        # draw.ellipse((plot_x - ps, plot_y - ps, plot_x + ps, plot_y + ps), fill=rgb_color)
        #draw.point((x, y), fill=rgb_color)
        draw.ellipse((Scale*x-2, Scale*y-2, Scale*x+2, Scale*y+2), fill=rgb_color)


    image.show()


    image.save(png_dir + "p={}z={}.png".format(p,"{:03}".format(z)))

8
